In [146]:
from dotenv import load_dotenv
import os 
load_dotenv()
api_key = os.environ["openai_api_key"]

Multi-Step Query Engine


In [147]:
from llama_index.core import VectorStoreIndex , SimpleDirectoryReader
from llama_index.llms.openai import OpenAI
from IPython.display import Markdown,display

In [148]:
# llm (gpt-4o-mini)

gpt4 = OpenAI(temperature=0.3,model='gpt-4o-mini')

# llm(gpt-4o-mini)
gpt4omini = OpenAI(temperature=0.4,model='gpt-4')

In [149]:
# load_documents
documents = SimpleDirectoryReader("data").load_data()
documents

[Document(id_='b1cf76e4-4d32-4ee3-bec5-dad7b8c0f167', embedding=None, metadata={'page_label': '1', 'file_name': 'ADV PYTHON.pdf', 'file_path': 'c:\\Users\\Nanhi Deo\\Desktop\\LangchainRag\\data\\ADV PYTHON.pdf', 'file_type': 'application/pdf', 'file_size': 1873460, 'creation_date': '2024-12-02', 'last_modified_date': '2024-06-10'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text=' \n  \nADVANCE PYTHON \n \nMOHAN REDDY ', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'),
 Document(id_='dcefabea-8c83-4b33-b13c-68706b61d139', embedding=None, metadata={'page_label': '2', 'file_name': 'ADV PYTHON.pdf', 'file_path': 'c

In [150]:
index = VectorStoreIndex.from_documents(documents,show_progress=True)
index

Parsing nodes:   0%|          | 0/137 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/137 [00:00<?, ?it/s]

In [151]:
# query_index
from llama_index.core.indices.query.query_transform.base import StepDecomposeQueryTransform

# gpt4
step_decompose_transform= StepDecomposeQueryTransform(llm=gpt4,verbose=True)

step_decompose_transform = StepDecomposeQueryTransform(
    llm = gpt4omini, verbose=True
)


In [152]:
index_summary = "Userd to answer question about the author"

In [153]:
# set Logging to DEBUG for more detailed outputs
from llama_index.core.query_engine import MultiStepQueryEngine

query_engine = index.as_query_engine(llm=gpt4)
query_engine = MultiStepQueryEngine(
    query_engine=query_engine,
    query_transform=step_decompose_transform,
    index_summary=index_summary,
)
response_gpt4 = query_engine.query(
    "what is python?, when did python start?"
    " Changes in and Disagreements With Accountants on Accounting and Financial Disclosure."
)
    

> Current query: what is python?, when did python start? Changes in and Disagreements With Accountants on Accounting and Financial Disclosure.
> New query: Who is the author of Python?
> Current query: what is python?, when did python start? Changes in and Disagreements With Accountants on Accounting and Financial Disclosure.
> New query: None


In [154]:
sub_qa = response_gpt4.metadata["sub_qa"]
tuples = [(t[0], t[1].response) for t in sub_qa]
print(tuples)

[('Who is the author of Python?', 'The author mentioned in the provided information is Mohan Reddy.')]


In [155]:
response_gpt4

Response(response='Python is a high-level programming language known for its simplicity and readability. It was first released in 1991 by its creator, Guido van Rossum.', source_nodes=[NodeWithScore(node=TextNode(id_='bf5a7d34-db29-44c8-8744-54509abf090a', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='\nQuestion: Who is the author of Python?\nAnswer: The author mentioned in the provided information is Mohan Reddy.', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=None), NodeWithScore(node=TextNode(id_='cfdd4555-04f4-4bef-8271-d2cc849ac7f6', embedding=None, metadata={'page_label': '1', 'file_name': 'ADV PYTHON.pdf', 'file_path': 'c:\\Users\\Nanhi Deo\\Desktop\\LangchainRag\\data\\ADV PYTHON.pdf', 'file_type': 'application/pdf', 'file_size': 1873460, 'creation_date': 

In [156]:
display(Markdown(f"{response_gpt4}"))

Python is a high-level programming language known for its simplicity and readability. It was first released in 1991 by its creator, Guido van Rossum.

In [157]:
sub_qa = response_gpt4.metadata["sub_qa"]
tuples = [(t[0],t[1].response) for t in sub_qa]
print(tuples)

[('Who is the author of Python?', 'The author mentioned in the provided information is Mohan Reddy.')]


In [158]:
response_gpt4 = query_engine.query(
    "in which city did the author found his first company , softwere? "
)

> Current query: in which city did the author found his first company , softwere? 
> New query: Who is the author who founded the company Softwere?
> Current query: in which city did the author found his first company , softwere? 
> New query: None


In [159]:
response_gpt4

Response(response='The author founded his first company, Softwere, in Seattle.', source_nodes=[NodeWithScore(node=TextNode(id_='d4e00440-c216-4eb6-a527-a6ed3f9e3e62', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='\nQuestion: Who is the author who founded the company Softwere?\nAnswer: The provided information does not mention an author who founded the company Softwere.', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=None), NodeWithScore(node=TextNode(id_='cfdd4555-04f4-4bef-8271-d2cc849ac7f6', embedding=None, metadata={'page_label': '1', 'file_name': 'ADV PYTHON.pdf', 'file_path': 'c:\\Users\\Nanhi Deo\\Desktop\\LangchainRag\\data\\ADV PYTHON.pdf', 'file_type': 'application/pdf', 'file_size': 1873460, 'creation_date': '2024-12-02', 'last_modified_date': '2024-06-1

In [160]:
display(Markdown(f"{response_gpt4}"))

The author founded his first company, Softwere, in Seattle.

In [161]:
query_engine = index.as_query_engine(llm=gpt4omini)
query_engine = MultiStepQueryEngine(
    query_engine=query_engine,
    query_transform=step_decompose_transform,
    index_summary=index_summary,
)

response_gpt3 = query_engine.query(
    ""
)

> Current query: 
> New query: Who is the author?
> Current query: 
> New query: None


In [162]:
response_gpt3

Response(response='The author is Mohan Reddy.', source_nodes=[NodeWithScore(node=TextNode(id_='64779708-d87c-4a2d-94c0-a4b67c06abb2', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='\nQuestion: Who is the author?\nAnswer: The author is Mohan Reddy.', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=None), NodeWithScore(node=TextNode(id_='181cb6a4-eba1-4c38-a321-3f16c310588c', embedding=None, metadata={'page_label': '18', 'file_name': 'form10-k.pdf', 'file_path': 'c:\\Users\\Nanhi Deo\\Desktop\\LangchainRag\\data\\form10-k.pdf', 'file_type': 'application/pdf', 'file_size': 616562, 'creation_date': '2024-11-27', 'last_modified_date': '2024-11-27'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_d

In [163]:
display(Markdown(f"{response_gpt3}"))

The author is Mohan Reddy.